In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import scipy.ndimage
import matplotlib.pyplot as plt
from numpy.ma.core import exp
from scipy.constants.constants import pi
from scipy import signal,misc, ndimage
import matplotlib.pyplot as plt
import scipy
import numpy as np
from scipy import stats

# Entrada dos dados subjetivos

In [2]:
df = pd.read_excel('data.xlsx')

df = df[[u'Imagem',   u'Estrutura', u'Avaliador 1', u'Avaliador 2']]
df.head()

,Imagem,Estrutura,Avaliador 1,Avaliador 2
0,Test_0_50_jp2_bone.png,Estruturas corticais e trabeculares dos ossos,2,2
1,Test_0_60_jp2_bone.png,Estruturas corticais e trabeculares dos ossos,4,5
2,Test_0_70_jp2_bone.png,Estruturas corticais e trabeculares dos ossos,3,5
3,Test_0_30_jpg_bone.png,Estruturas corticais e trabeculares dos ossos,2,4
4,Test_0_40_jpg_bone.png,Estruturas corticais e trabeculares dos ossos,4,4


# Implementação das métricas de qualidade visual

In [3]:
def epm(r, d):
    #gradient direction similarity - reference
    r = scipy.misc.imread(r, mode='L')
     
    Sx = np.array([[-1,0,1],[-2,0,2],[-1,0, 1]])*(1/4.0)
    Sy = np.array([[1,2, 1],[0,0,0],[-1,-2,-1]])*(1/4.0)
    
    Gxr = scipy.ndimage.filters.convolve(r,Sx)
    Gyr = scipy.ndimage.filters.convolve(r,Sy)
     
    #gradient direction similarity - distorted
    d = scipy.misc.imread(d, mode='L')
     
    Gxd = scipy.ndimage.filters.convolve(d,Sx)
    Gyd = scipy.ndimage.filters.convolve(d,Sy)

    #gradiente magnitude weighting map - reference
     
    gr = (1/1.118)*np.round(np.sqrt(np.power(Gxr,2.0) + np.power(Gyr,2.0)))
    gd = (1/1.118)*np.round(np.sqrt(np.power(Gxd,2.0) + np.power(Gyd,2.0)))

    ar = np.arctan(Gyr/(Gxr+0.001))
    ad = np.arctan(Gyd/(Gxd+0.001))

    #
    c=1
    delta_g = np.copy(gr)
    for i in range(0, len(gr)):
        for j in range(0,len(gr[0])):
            if gr[i,j]>gd[i,j]:
                delta_g[i,j] = (gd[i,j]+c)/(gr[i,j]+c)
            if gd[i,j]>=gr[i,j]:
                delta_g[i,j] = (gr[i,j]+c)/(gd[i,j]+c)
    delta_a = (1/180.0)* np.abs((np.abs(ar-ad))-180)


    Q_g =(1.03688316740124)/(1+(np.exp(-11.0*(delta_g-0.7))))
    Q_a = (1.00822974704902)/(1+(np.exp(-24.0*(delta_a-0.8))))


    Q_ab = np.sqrt((Q_g*Q_a))
    
    hist = np.zeros(int(np.max(gr))+1)
    
    for i in range(0, len(r)):
        for j in range(0, len(r[0])):
            hist[int((gr[i,j]))] = hist[int((gr[i,j]))]+1
    ga = np.copy(gr)
    for i in range(0, len(gr)):
        for j in range(0, len(gr[0])):
            ga[i,j] = hist[int((gr[i,j])-1)]
    w = -np.log2(ga+0.00001)

    Q = np.nansum((Q_ab*w))/np.nansum(w)
    return Q
def gmw_gdsim(r,d):
    #gradient direction similarity - reference
    r = scipy.misc.imread(r, mode='L')
    d = scipy.misc.imread(d, mode='L')
    Sx = np.array([[1,0,-1],[2,0,-2],[1,0, -1]])
    Sy = np.array([[1,2,1],[0,0,0],[-1,-2, -1]])

    Gxr = scipy.ndimage.filters.convolve(r,Sx)
    Gyr = scipy.ndimage.filters.convolve(r,Sy)

    GDr = np.arctan(Gxr/(Gyr+0.001))

    #gradient direction similarity - distorted


    Gxd = scipy.ndimage.filters.convolve(d,Sx)
    Gyd = scipy.ndimage.filters.convolve(d,Sy)

    GDd = np.arctan(Gxd/(Gyd+0.001))

    #gradient direction delta

    GD = np.abs((GDr - GDd))
    GDSIM = np.cos(GD)

    #gradiente magnitude weighting map - reference

    GMr = np.sqrt(np.power(Gxr,2.0) + np.power(Gyr,2.0))
    GMd = np.sqrt(np.power(Gxd,2.0) + np.power(Gyd,2.0))

    GMc = (GMr + GMd)/2
    GMth = 100.0
    GMcnew = np.copy(GMc)

    for i in range(0, len(GMc)):
        for j in range(0, len(GMc[0])):
            if GMc[i,j] <= GMth:
                GMcnew[i,j] = 0
            if GMc[i,j]> GMth:
                GMcnew[i,j] = GMc[i,j] - GMth
    GMW = scipy.ndimage.filters.gaussian_filter(GMcnew,2)

    GMW_GDSIM = np.sum(GMW*GDSIM)/np.sum(GMW)
    return GMW_GDSIM

def msim(img_mat_1, img_mat_2):
    img_mat_1 = scipy.misc.imread(img_mat_1, mode='L')
    img_mat_2 = scipy.misc.imread(img_mat_2, mode='L')
    std =3
    L = 6*std +1 
    window = signal.gaussian(L, std) 
    

    
    w = window.reshape((L,1))
    wt = np.transpose(w)
    h = w * wt
    soma = np.sum(h)
    h = h/soma
    
    
    img_mat_1=img_mat_1.astype(np.float)
    img_mat_2=img_mat_2.astype(np.float)

    
    img_mat_1_sq=img_mat_1**2
    img_mat_2_sq=img_mat_2**2
    img_mat_12=img_mat_1*img_mat_2
    
    
    img_mat_mu_1=scipy.ndimage.filters.convolve(img_mat_1,h)
    img_mat_mu_2=scipy.ndimage.filters.convolve(img_mat_2,h)
        
    
    img_mat_mu_1_sq=img_mat_mu_1**2
    img_mat_mu_2_sq=img_mat_mu_2**2
    img_mat_mu_12=img_mat_mu_1*img_mat_mu_2
    
    
    img_mat_sigma_1_sq=scipy.ndimage.filters.convolve(img_mat_1_sq,h)
    img_mat_sigma_2_sq=scipy.ndimage.filters.convolve(img_mat_2_sq,h)
    
    
    img_mat_sigma_12=scipy.ndimage.filters.convolve(img_mat_12,h)

    
    img_mat_sigma_1_sq=img_mat_sigma_1_sq-img_mat_mu_1_sq
    img_mat_sigma_2_sq=img_mat_sigma_2_sq-img_mat_mu_2_sq
    img_mat_sigma_12=img_mat_sigma_12-img_mat_mu_12;

    

    c_1=6.5025
    c_2=58.5225
    
    
    l=255
    k_1=0.01
    c_1=(k_1*l)**2
    k_2=0.03
    c_2=(k_2*l)**2
    
    
    num_ssim=(2*img_mat_mu_12+c_1)*(2*img_mat_sigma_12+c_2)
    
    den_ssim=(img_mat_mu_1_sq+img_mat_mu_2_sq+c_1)*\
    (img_mat_sigma_1_sq+img_mat_sigma_2_sq+c_2)
    
    ssim_map=num_ssim/den_ssim
    
    value = np.average(ssim_map)
    return value

# Nome das imagens de referência

In [4]:
nome_imagens = df['Imagem'].unique()
media = pd.DataFrame(columns=['Imagem', 'Avaliador 1', 'Avaliador 2'])

for w in nome_imagens:
    nome_split = w.split('_')
    if nome_split[-1] == u'soft.png':
        a =  df[df['Imagem'] == w]
        media = media.append({'Imagem': a['Imagem'].values[1], 'Avaliador 1': round(np.mean(a['Avaliador 1'].values)), 'Avaliador 2' : round(np.mean(a['Avaliador 2'].values))}, ignore_index=True)
    if nome_split[-1] == u'bone.png':
        media = media.append(df[df['Imagem'] == w][['Imagem', 'Avaliador 1', 'Avaliador 2']])
media.head()

,Imagem,Avaliador 1,Avaliador 2
0,Test_0_50_jp2_bone.png,2,2
1,Test_0_60_jp2_bone.png,4,5
2,Test_0_70_jp2_bone.png,3,5
3,Test_0_30_jpg_bone.png,2,4
4,Test_0_40_jpg_bone.png,4,4


In [5]:
ref = pd.DataFrame(columns=['Imagem', 'Avaliador 1', 'Avaliador 2', 'Imagem de referencia'])
for i in range(0,42,7):
    a = media.iloc[i:i+7, :]
    a['Imagem de referencia'] = a.iloc[-1,0]
    ref = ref.append(a)
ref.head()

c:\users\fitec\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Imagem,Avaliador 1,Avaliador 2,Imagem de referencia
0,Test_0_50_jp2_bone.png,2,2,Orig_0_bone.png
1,Test_0_60_jp2_bone.png,4,5,Orig_0_bone.png
2,Test_0_70_jp2_bone.png,3,5,Orig_0_bone.png
3,Test_0_30_jpg_bone.png,2,4,Orig_0_bone.png
4,Test_0_40_jpg_bone.png,4,4,Orig_0_bone.png


In [6]:
ref["JPG"] = [ref["Imagem"].values[i].split('_')[-2] for i in range(len(ref))]

In [7]:
g = ref['JPG']
for i in range(3):
    g.replace(str(i),'jpeg,jp2', inplace=True)
ref['JPG'] = g

In [8]:
diretorio = "C:/Users/fitec/OneDrive/Documentos/Dissertacao/imagens_teste/"
ref['epm'] = list(map(epm, diretorio+ref["Imagem de referencia"].values, diretorio+ref["Imagem"].values))
ref['gmw_gdsim'] = list(map(gmw_gdsim, diretorio+ref["Imagem de referencia"].values, diretorio+ref["Imagem"].values))
ref['mssim'] = list(map(msim, diretorio+ref["Imagem de referencia"].values, diretorio+ref["Imagem"].values))


c:\users\fitec\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\fitec\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  if sys.path[0] == '':
c:\users\fitec\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:58: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
c:\users\fitec\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:59: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated i

In [9]:
ref.head()

,Imagem,Avaliador 1,Avaliador 2,Imagem de referencia,JPG,epm,gmw_gdsim,mssim
0,Test_0_50_jp2_bone.png,2,2,Orig_0_bone.png,jp2,0.909991,0.808538,0.998260
1,Test_0_60_jp2_bone.png,4,5,Orig_0_bone.png,jp2,0.919882,0.817677,0.998630
2,Test_0_70_jp2_bone.png,3,5,Orig_0_bone.png,jp2,0.941139,0.853879,0.999162
3,Test_0_30_jpg_bone.png,2,4,Orig_0_bone.png,jpg,0.888369,0.789333,0.994497
4,Test_0_40_jpg_bone.png,4,4,Orig_0_bone.png,jpg,0.886112,0.790413,0.904367


In [10]:
ref['mos'] = [np.floor(np.mean([ref['Avaliador 1'].values[i], ref['Avaliador 2'].values[i]])) for i in range(len(ref))]

In [11]:
ref['taxa'] = [i.split('_')[-3] for i in ref['Imagem'].values]
ref.replace('Orig', 100, inplace=True)

In [12]:
ref.head()

,Imagem,Avaliador 1,Avaliador 2,Imagem de referencia,JPG,epm,gmw_gdsim,mssim,mos,taxa
0,Test_0_50_jp2_bone.png,2.0,2.0,Orig_0_bone.png,jp2,0.909991,0.808538,0.998260,2.0,50
1,Test_0_60_jp2_bone.png,4.0,5.0,Orig_0_bone.png,jp2,0.919882,0.817677,0.998630,4.0,60
2,Test_0_70_jp2_bone.png,3.0,5.0,Orig_0_bone.png,jp2,0.941139,0.853879,0.999162,4.0,70
3,Test_0_30_jpg_bone.png,2.0,4.0,Orig_0_bone.png,jpg,0.888369,0.789333,0.994497,3.0,30
4,Test_0_40_jpg_bone.png,4.0,4.0,Orig_0_bone.png,jpg,0.886112,0.790413,0.904367,4.0,40


In [13]:
ref.to_csv(index=False)

'Imagem,Avaliador 1,Avaliador 2,Imagem de referencia,JPG,epm,gmw_gdsim,mssim,mos,taxa\r\nTest_0_50_jp2_bone.png,2.0,2.0,Orig_0_bone.png,jp2,0.9099914117064495,0.8085378604876444,0.9982600660478493,2.0,50\r\nTest_0_60_jp2_bone.png,4.0,5.0,Orig_0_bone.png,jp2,0.9198816630095655,0.8176766701576446,0.9986298909459106,4.0,60\r\nTest_0_70_jp2_bone.png,3.0,5.0,Orig_0_bone.png,jp2,0.9411388066612301,0.8538791204506927,0.9991618986660449,4.0,70\r\nTest_0_30_jpg_bone.png,2.0,4.0,Orig_0_bone.png,jpg,0.8883687142420735,0.7893330768307738,0.9944971205605834,3.0,30\r\nTest_0_40_jpg_bone.png,4.0,4.0,Orig_0_bone.png,jpg,0.8861117101223871,0.7904132940310025,0.9043673060408659,4.0,40\r\nTest_0_50_jpg_bone.png,2.0,5.0,Orig_0_bone.png,jpg,0.8886763731056053,0.7921911914641641,0.9958700476787578,3.0,50\r\nOrig_0_bone.png,4.0,5.0,Orig_0_bone.png,"jpeg,jp2",1.0,1.0,1.0,4.0,100\r\nTest_0_50_jp2_soft.png,5.0,2.0,Orig_0_soft.png,jp2,0.8648073205316629,0.8166718889873162,0.9760124561038214,3.0,50\r\nTest_0_60_j